In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

import json
import codecs

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
# %matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Import data
with codecs.open('Total_WOS.json', 'r', 'utf-8-sig') as json_file:  
    df= pd.DataFrame(json.load(json_file))	
df.head()

# Convert to list
data = df.values.tolist()
pprint(data[:1])

[["First to respond, last to leave: Communities' roles and resilience across "
  "the '4Rs'",
  'Abstract\n'
  'Shortly after the second major earthquake in Canterbury, New Zealand in '
  '2011, an inventory of the activities and initiatives of community groups '
  'and NGOs was conducted. Another two inventories were undertaken in 2012 and '
  '2013. Analysis of these three inventories reveals some significant changes '
  "in both the types of activities being undertaken and the 'communities' "
  'responsible for them. The inventories document a rapid increase in the '
  'number of community groups and non-governmental organisations playing a '
  'role in recovery. An important finding is that most of those active almost '
  '4 years after the first earthquake were actually in operation before the '
  'earthquakes and have since layered recovery work over their core business. '
  'In building resilience to all hazards, we therefore argue for better '
  'recognition of the potential ro

In [2]:

# Tokenize words
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])


[['first', 'to', 'respond', 'last', 'to', 'leave', 'communities', 'roles', 'and', 'resilience', 'across', 'the', 'rs', 'abstract', 'nshortly', 'after', 'the', 'second', 'major', 'earthquake', 'in', 'canterbury', 'new', 'zealand', 'in', 'an', 'inventory', 'of', 'the', 'activities', 'and', 'initiatives', 'of', 'community', 'groups', 'and', 'ngos', 'was', 'conducted', 'another', 'two', 'inventories', 'were', 'undertaken', 'in', 'and', 'analysis', 'of', 'these', 'three', 'inventories', 'reveals', 'some', 'significant', 'changes', 'in', 'both', 'the', 'types', 'of', 'activities', 'being', 'undertaken', 'and', 'the', 'communities', 'responsible', 'for', 'them', 'the', 'inventories', 'document', 'rapid', 'increase', 'in', 'the', 'number', 'of', 'community', 'groups', 'and', 'non', 'governmental', 'organisations', 'playing', 'role', 'in', 'recovery', 'an', 'important', 'finding', 'is', 'that', 'most', 'of', 'those', 'active', 'almost', 'years', 'after', 'the', 'first', 'earthquake', 'were', 'a

In [3]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])



['first', 'to', 'respond', 'last', 'to', 'leave', 'communities', 'roles', 'and', 'resilience', 'across', 'the', 'rs', 'abstract', 'nshortly', 'after', 'the', 'second', 'major', 'earthquake', 'in', 'canterbury_new_zealand', 'in', 'an', 'inventory', 'of', 'the', 'activities', 'and', 'initiatives', 'of', 'community', 'groups', 'and', 'ngos', 'was', 'conducted', 'another', 'two', 'inventories', 'were', 'undertaken', 'in', 'and', 'analysis', 'of', 'these', 'three', 'inventories', 'reveals', 'some', 'significant', 'changes', 'in', 'both', 'the', 'types', 'of', 'activities', 'being', 'undertaken', 'and', 'the', 'communities', 'responsible', 'for', 'them', 'the', 'inventories', 'document', 'rapid', 'increase', 'in', 'the', 'number', 'of', 'community', 'groups', 'and', 'non_governmental_organisations', 'playing', 'role', 'in', 'recovery', 'an', 'important', 'finding', 'is', 'that', 'most', 'of', 'those', 'active', 'almost', 'years', 'after', 'the', 'first', 'earthquake', 'were', 'actually', 'in

In [4]:

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['first', 'respond', 'last', 'leave', 'community', 'role', 'resilience', 'nshortly', 'second', 'major', 'earthquake', 'canterbury', 'inventory', 'activity', 'initiative', 'community', 'group', 'ngo', 'conduct', 'inventory', 'undertake', 'analysis', 'inventory', 'reveal', 'significant', 'change', 'type', 'activity', 'undertake', 'community', 'responsible', 'inventory', 'document', 'rapid', 'increase', 'number', 'community', 'group', 'organisation', 'play', 'role', 'recovery', 'important', 'finding', 'active', 'almost', 'year', 'first', 'earthquake', 'actually', 'operation', 'earthquake', 'layered', 'recovery', 'work', 'core', 'business', 'building', 'resilience', 'hazard', 'therefore', 'argue', 'well', 'recognition', 'potential', 'role', 'different', 'community', 'play', 'recovery', 'phase', 'disaster', 'risk', 'reduction', 'role', 'range', 'administer', 'first', 'aid', 'co', 'produce', 'knowledge', 'underpinning', 'risk', 'management', 'strategy', 'action', 'argue', 'different', 'type

In [5]:

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 2), (10, 1), (11, 2), (12, 1), (13, 1), (14, 1), (15, 1), (16, 7), (17, 1), (18, 2), (19, 2), (20, 3), (21, 1), (22, 3), (23, 1), (24, 1), (25, 3), (26, 2), (27, 1), (28, 1), (29, 1), (30, 1), (31, 4), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 2), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 2), (47, 1), (48, 2), (49, 2), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 5), (56, 2), (57, 1), (58, 3), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 3), (66, 4), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 2), (74, 1), (75, 1), (76, 2), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1)]]


In [6]:

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=8, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.039*"risk" + 0.030*"flood" + 0.022*"disaster" + 0.021*"vulnerability" + '
  '0.020*"hazard" + 0.015*"area" + 0.015*"impact" + 0.012*"natural" + '
  '0.012*"event" + 0.011*"study"'),
 (1,
  '0.037*"model" + 0.018*"use" + 0.017*"earthquake" + 0.013*"method" + '
  '0.013*"base" + 0.012*"damage" + 0.010*"result" + 0.010*"study" + '
  '0.010*"area" + 0.010*"keyword"'),
 (2,
  '0.062*"information" + 0.060*"landslide" + 0.026*"medium" + 0.024*"datum" + '
  '0.019*"rainfall" + 0.017*"system" + 0.014*"use" + 0.011*"debris_flow" + '
  '0.011*"fire" + 0.011*"technology"'),
 (3,
  '0.094*"earthquake" + 0.031*"school" + 0.028*"education" + 0.022*"child" + '
  '0.016*"student" + 0.015*"year" + 0.014*"disaster" + 0.013*"study" + '
  '0.012*"post" + 0.012*"stress"'),
 (4,
  '0.097*"infrastructure" + 0.063*"sector" + 0.036*"critical" + '
  '0.033*"indirect" + 0.033*"economic" + 0.032*"business" + '
  '0.027*"interdependence" + 0.025*"loss" + 0.023*"direct" + 0.022*"impact"'),
 (5,
  '0.058*"d

In [7]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.719932317362921

Coherence Score:  0.3327608157875921


In [8]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis
pyLDAvis.save_html(vis, 'LDAGensim_Total_WOS.html')

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
